# COVID 19 - Image Enhancement

In this notebook, we will see the existing method for x-rays image enhancement.
The idea of this notebook, is do get a better visualization of the x-rays images, which could be useful for doctors to see possible disease or for deep learning techniques to preprocess the input images.

If you any remark, feel free to leave a comment. Good reading.

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
import glob
import pydicom

from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image

import cv2 as cv

import random 

random.seed(42)

In [ ]:
TARGET_SIZE = 512

def dicom2array(path, voi_lut = True, fix_monochrome = True):
    """
    Transform a dicom file to an array.
    
    - path : path of the dicom file
    - voi_lut : Apply VOI LUT transformation
    - fix_monochrome : Indicate if we fix the pixel value for specific files.
    
    VOI LUT (Value of Interest - Look Up Table) : The idea is to have a larger representation of the data.
    Since, dicom files have larger pixel display range than usuall pictures. The idea is to keep a larger representation in order ot better see the subtle differences.
    
    Fix Monochrome : Some images have MONOCHROME1 interpretation. Which means that higher pixel values corresponding to the dark instead of the white.
    """
    dicom = pydicom.read_file(path)
    
    # Apply the VOI LUT
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # Fix the representation
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
    data = data / np.max(data)
    
    data = data * 255
    
    data = data.astype(np.uint8)
    
    return data

def img_vizualisation(imgs, nb_samples = 5):
    
    fig, axes = plt.subplots(nrows=nb_samples // 5, ncols=min(5, nb_samples), figsize=(min(5, nb_samples) * 4, 4 * (nb_samples // 5)))
    i = 0
    for img in imgs:
        axes[i // 5, i % 5].imshow(np.array(img), cmap=plt.cm.gray, aspect='auto')
        axes[i // 5, i % 5].axis('off')
        i += 1
    fig.show()    


# Sample visualization

To see the different techniques, we will have a sample set that will serve as a reference.

In [ ]:
TRAIN_PATH = "../input/siim-covid19-detection/train/"

paths = glob.glob(TRAIN_PATH + "*/*/*.dcm")

In [ ]:
sampled_path = random.sample(paths, 10)

samples = []
for path in sampled_path:
    img = dicom2array(path)
    samples.append(img)

In [ ]:
img_vizualisation(samples, 10)

# Histogram equalization

Histogram equalization is a method that **increases the global contrast of images**, especially when the image is represented by a narrow range of intensity values. With this method, we can adjust the pixel intensity in order to obtain a better distribution on the histogram using the full range of intensities evenly. This allows for areas of lower local constrast to gain higher contrast. 

For radiography, this could be usefull, to have a better view of bone structure in x-ray images. A disadvantge of this method is that it is indiscriminate. It may increase the contrast of background noise, while decreasing the usable signal.

> https://en.wikipedia.org/wiki/Histogram_equalization

### Visualization of histogram equalization transformation

To have a better idea of the result of histogram equalization, we could take the top right picture in our reference set. With this picture, it's really difficult to see the content of the image. If we have a look at the histogram of value of this picture (see below), we see that the pixel values are squeezes through 0 and 75. With histogram equalization, we can resample the values, in order to use the full range of pixels intensities.

In [ ]:
img_example = samples[4]
img_example_hist = cv.equalizeHist(img_example)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.imshow(img_example, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title("Original image")

ax2.imshow(img_example_hist, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title("Histogram Equalization applied on the original image")

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(img_example_hist.flatten(), 256, [0, 256])
ax2.set_title("Histogram Equalization applied on the original image")

fig.show()

### Visualization on the sample data

In [ ]:
import scipy.misc

equalized_samples = []
for sample in samples:
    img = cv.equalizeHist(sample)
    equalized_samples.append(img)
    
img_vizualisation(equalized_samples, 10)

# CLAHE (Contrast Limited Adaptive Histogram Equalization) 

The first histogram equalization we just saw, considers the global contrast of the image. In many cases, it is not a good idea. 
We could have part of the image lost due to over-brightness.

So, to solve this problem, **adaptive histogram equalization** is used. To apply this method, image is divided into small blocks called "tiles". Then each of these blocks are histogram equalized as usual. So in a small area, histogram would confine to a small region (unless there is noise). If noise is there, it will be amplified. To avoid this, **contrast limiting** is applied. If any histogram bin is above the specified contrast limit, those pixels are clipped and distributed uniformly to other bins before applying histogram equalization. After equalization, to remove artifacts in tile borders, bilinear interpolation is applied.

### Visualization of CLAHE

In [ ]:
clahe = cv.createCLAHE(clipLimit=40.0, tileGridSize=(8,8))

img_example = samples[9]
img_example_clahe = clahe.apply(img_example)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.imshow(img_example, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title("Original image")

ax2.imshow(img_example_clahe, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title("CLAHE applied on the original image")

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(img_example_clahe.flatten(), 256, [0, 256])
ax2.set_title("CLAHE applied on the original image")

fig.show()

### Visualization on the sample data

In [ ]:
clahe_samples = []

clahe = cv.createCLAHE(clipLimit=40.0, tileGridSize=(8,8))

for sample in samples:
    img = clahe.apply(sample)
    clahe_samples.append(img)
    
img_vizualisation(clahe_samples, 10)

### CLAHE with different parameters

In [ ]:
clahe_samples_2 = []

clahe = cv.createCLAHE(clipLimit=20.0, tileGridSize=(15, 15))

for sample in samples:
    img = clahe.apply(sample)
    clahe_samples_2.append(img)
    
img_vizualisation(clahe_samples_2, 10)

## Contrast Enhancement of Medical X-Ray ImageUsing Morphological Operators with OptimalStructuring Element

Paper : Contrast Enhancement of Medical X-Ray ImageUsing Morphological Operators with OptimalStructuring Element
> https://arxiv.org/pdf/1905.08545.pdf

The idea of this paper is to apply a combination of Top-hat and Bottom-hat Transform. 

- Top Hat : It is the difference between input image and Opening of the image. 
- Opening : Erosion followed by dilatation

- Black Hat : It is the difference between the closing of the input image and input image. 
- Closing : Dilation followed by Erosion.

I have made two implementations, one with skimage library using a disk as the footprint used for top and black hat. And another implementation with opencv using an ellipse. The opencv method is faster than the skimage. 

Regarding the result, I am not really satisfied by this method (I maybe miss something in the paper). I let you see the result.

In [ ]:
import skimage.morphology
from skimage.morphology import disk

img_example = samples[9]

footprint = disk(15)

top = skimage.morphology.white_tophat(img_example, footprint)
bottom = skimage.morphology.black_tophat(img_example, footprint)

output = img_example + top - bottom


compare = np.concatenate((img_example, output), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)
plt.show()

In [ ]:
img_example = samples[9]

# The size of the structured element has impact 
filterSize = (15, 15)
kernel = cv.getStructuringElement(cv.MORPH_RECT, filterSize) # MORPH_ELLIPSE


tophat_img = cv.morphologyEx(img_example, cv.MORPH_TOPHAT, kernel)
bothat_img = cv.morphologyEx(img_example, cv.MORPH_BLACKHAT, kernel) # Black --> Bottom

output = img_example + tophat_img - bothat_img

compare = np.concatenate((img_example, output), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)
plt.show()

In [ ]:
hat_samples = []

kernel = cv.getStructuringElement(cv.MORPH_RECT, (15, 15)) # MORPH_ELLIPSE

for sample in samples:

    tophat = cv.morphologyEx(sample, cv.MORPH_TOPHAT, kernel)
    bothat = cv.morphologyEx(sample, cv.MORPH_BLACKHAT, kernel)
    img = sample + tophat - bothat

    hat_samples.append(img)
    
img_vizualisation(hat_samples, 10)

## Change the approach

I'm really not satisfy with the last approach. I think that maybe by applying histogram equalization first and then applying it the morphological operator could be interesting.

In [ ]:
img_example = equalized_samples[9]

kernel = cv.getStructuringElement(cv.MORPH_RECT, (15, 15)) # MORPH_ELLIPSE

tophat_img = cv.morphologyEx(img_example, cv.MORPH_TOPHAT, kernel)
bothat_img = cv.morphologyEx(img_example, cv.MORPH_BLACKHAT, kernel) # Black --> Bottom

output = img_example + tophat_img - bothat_img

compare = np.concatenate((img_example, output), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(output.flatten(), 256, [0, 256])
ax2.set_title("This approach on the original image")

fig.show()

In [ ]:
hat_samples = []

kernel = cv.getStructuringElement(cv.MORPH_RECT, (15, 15)) # MORPH_ELLIPSE

for sample in equalized_samples:

    tophat = cv.morphologyEx(sample, cv.MORPH_TOPHAT, kernel)
    bothat = cv.morphologyEx(sample, cv.MORPH_BLACKHAT, kernel)
    img = sample + tophat - bothat

    hat_samples.append(img)
    
img_vizualisation(hat_samples, 10)

# Noise reduction

When we realize a x-rays images, we could had some noise during the process. Before doing image enhancement, it could be interesting to reduce/remove the noise present in our image. In this section, we are going to see two filter : the median filter and the DCT filter.

## Median filter 

The first method that we are going to use is the median filter. For each pixel value on our image, we will select the median value from the values next to the current pixel. In order to do that, we will pass over our image a box (for example : a box of 5x5). This will allow us to remove abnormal value.

In [ ]:
img_example = samples[0]
# Note : ksize need to be an odd number.
noised_samples_example = cv.medianBlur(img_example, 5)

compare = np.concatenate((img_example, noised_samples_example), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(noised_samples_example.flatten(), 256, [0, 256])
ax2.set_title("Noise reduction on the original image")

fig.show()

## DCT-based filter

The idea of DCT (Discrete Cosinus Transform) based filter is to decompose a signal as a sum of cosines function. The DCT is the same idea of Fourier transformation where we decompose a signal into frequencies. The DCT is well used for data compression. But, here we are going to use it for noise reduction.

First, we need to understand that an image can be decomposed as signals from the two dimensions of our image. Signals from the x-axis and signals form the y-axis. Based on that, we can decompose our image by decomposing each signal of our image. Each decomposition tries to get the core of the signal. It tries to get the best continues signals possible.

Thus, if for a signal, we have an abnormal value for one given point (noise), by decomposing the signal we try to get the core of the signal. Then, this abnormal value will .

Finally, with the decomposition of our image, we can reconstruct the image by applying the inverse function of DCT. By doing that, we get an image similar from the original one but with no abnormals values.  

In [ ]:
# https://stackoverflow.com/questions/7110899/how-do-i-apply-a-dct-to-an-image-in-python
from scipy.fftpack import dct, idct

def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho')  

def dtc_transform(img):
    return idct2(dct2(img))

img_example = samples[0]
img_idct = dtc_transform(img_example)

print("Check if the image are similar :", np.allclose(img_example, img_idct))

compare = np.concatenate((img_example, img_idct), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(img_idct.flatten(), 256, [0, 256])
ax2.set_title("Noise reduction on the original image")

fig.show()

In [ ]:
dct_samples = []

for sample in samples:
    img = dtc_transform(sample)
    dct_samples.append(img)
    
img_vizualisation(dct_samples, 10)

# Combine the different approaches

As we are dealing with grayscale image, it could be interesting to add different approaches and merge them in a single image in order to create a color image. The main advantage of this, is as we are usually using pretrained model, almost all of them use 3 channel. So, why not given additional information instead of just duplicate the channel.

In this proposition, I will keep the original image in the first channel and put in the two remaining channels transformed images from the original one.

In [ ]:
channel_1 = samples[9]
channel_2 = clahe_samples_2[9]
channel_3 = hat_samples[9]

# Merge all channel to create our image
output = np.dstack((channel_1, channel_2, channel_3))

reference = np.dstack((channel_1, channel_1, channel_1))

# See the difference
compare = np.concatenate((reference, output), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare)
plt.show()

In [ ]:
multi_channel_samples = []

for i in range(len(samples)):
    channel_1 = samples[i]
    channel_2 = clahe_samples_2[i]
    # channel_2 = clahe_samples[i]
    channel_3 = hat_samples[i]
    
    out_img = np.dstack((channel_1, channel_2, channel_3))
    
    multi_channel_samples.append(out_img)
    
img_vizualisation(multi_channel_samples, 10)

# Conclusion

In this notebook, we see some techniques that could be useful for image enhancement. 

Thanks for your reading and your feedback. Don't hesitate if you have questions. 

If you want to go further, there are other methods that I have not mentioned. Do not hesitate to go and see if you are interested.

- MAHE (Median adaptive histogram equalization)
- Multiple Morphological Gradient (mMG)
- Wieners filter

## References 

- https://en.wikipedia.org/wiki/Histogram_equalization
- https://docs.opencv.org/master/d5/daf/tutorial_py_histogram_equalization.html
- https://www.ndt.net/article/icem2004/papers/64/64.htm
- https://medium.com/@florestony5454/median-filtering-with-python-and-opencv-2bce390be0d1
